# Lab4 - Part 2: Using a deployed Watson Machine Learning model from a python notebook

This is a 'stretch' lab to complement the Watson ML model training and deployment, we show how to invoke the deployed Watson ML scoring from a Jupyter Notebook

### Gathering information from the WML environment and deployed model
You will have to get out of this notebook, possibly in a separate tab, to gather the following information, or get it from your Notepad if you have correctly followed the previous Lab instructions.

#### Watson Machine Learning Credentials:
* You will need to retrieve your ML service credentials:
  * These are available form the Watson Data Platform `Services/Compute Services` menu
  * Selecting `Manage in IBM  Cloud` will take you to the service page where you can copy the credentials from
* Paste the credentials in the code cell below and execute it

In [ ]:
### REPLACE the code below with your own credentials placed in the wml_credentials variable

wml_credentials={
  "url": "https://us-south.ml.cloud.ibm.com",
  "access_key": "D2HOPDsxhZE310Dc5dP9p8xO3M3o5VH7bChSJuOGJeMI",
  "username": "5ac9555e-631b-486c-b484-7d6e556c97e3",
  "password": "78c4c71e-5eae-4ccd-926d-d8a40c264d18",
  "instance_id": "76075c55-c628-45c5-9de4-143fbd42238d"
}

#### Watson Machine Learning deployed model boilerplate code:
* Navigate back to you Project's Assets, down to `Models` section
* Open the deployed model, navigate to the `Implementation` tab
* Copy the Python sample code from your Deployed ML instance, and paste to _partially_ **replace** the code cell below:
  * Comment-out or remove the wml_credentials={....} lines, as they are already set in the cell above
  * Set the `payload_scoring` back to: 
``` python
payload_scoring = {"fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION"], "values": [["M",23,"Married","Professional"],["F",27,"Single","Professional"]]}
```
* Execute the code cell, this will run the scoring against the two candidates specified in `payload_scoring`

In [ ]:
import urllib3, requests, json

# retrieve your wml_service_credentials_username, wml_service_credentials_password, and wml_service_credentials_url from the
# Service credentials associated with your IBM Cloud Watson Machine Learning Service instance


headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION"], "values": [["M",23,"Married","Professional"],["F",27,"Single","Professional"]]}

response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/v3/wml_instances/76075c55-c628-45c5-9de4-143fbd42238d/deployments/f8348682-bd48-4109-896b-05c2112fad7d/online', json=payload_scoring, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))


### Display scoring results
As a final example, we will load the result scoring into a panda dataframe and display it with Brunel

In [ ]:
# Create a panda DataFrame with the values and fields
import pandas as pd
jscore=json.loads(response_scoring.text)
dfsc=pd.DataFrame(data=jscore['values'],columns=jscore['fields'])
dfsc

In [ ]:
import brunel
%brunel data('dfsc') bar x(AGE) y(probability) color(GENDER)

Feel free to experiment with other visualizations or input data